In [1]:
from pymongo import MongoClient
import psycopg2
import pickle
import re
import numpy as np
import pandas as pd
from datetime import datetime
from geopy.geocoders import Nominatim
geolocator = Nominatim()
import time
import sys
sys.path.insert(0, './text2num/')
import text2num

###Load pickle file

In [2]:
with open("ufo_nosummary.pkl","r") as pf:
    d = pickle.load(pf)

###Make into pandas df

In [3]:
df = pd.DataFrame([d["city"],d["title"],d["shape"],d["duration"],d["state"],d["link"],d["date"]])
df = df.transpose()
df.columns = ['city','title','shape','duration','state','link','date']

###Shapes

In [192]:
def shapes(x):
    dic = {"triangular":"triangle","triange":"triangle","round":"circle",
           "changed":"changing","pyramid":"triangle","flare":"light",
           "dome":"disk","hexagon":"other","crescent":"other","delta":"chevron"}
    if x in dic.keys():
        return dic[x]
    else:
        return x

In [193]:
df['shape'] = df['shape'].map(lambda x: shapes(x.lower()) if x else "unknown")

###Date

In [7]:
def standardize_dt(dateStr):
    for fmt in ["%m/%d/%y","%m/%d/%y %H:%M"]:
        try:
            return datetime.strptime(dateStr, fmt)
        except ValueError:
            continue

In [8]:
#not using

def date_only(x):
    reg_date = "\d{1,2}\/\d{1,2}\/\d{2}"
    result = re.findall(reg_date,str(x))
    if len(result)>0:
        return datetime.strptime(result[0], "%m/%d/%y")
    else:
        return "None"
def time_only(x):
    reg_time = "\d{2}\:\d{2}"
    result = re.findall(reg_time,str(x))
    if len(result)>0:
        return datetime.strptime(result[0], "%H:%M")
    else:
        return "None"

In [8]:
df['date_datetime'] = df['date'].map(lambda x: standardize_dt(str(x)))

###Day of week

In [198]:
df['day']=df['date_datetime'].map(lambda x: x.weekday())

###Duration

In [98]:
def convertTime(elem):  
    t = 1
    lower = elem.lower()
    #TODO: learn regex
    split = elem.replace("-"," ").replace("s"," s").replace("h"," h").replace("m"," m").replace(":00"," min ").replace("00:"," ").replace("~"," ").replace("_"," ").replace("+"," ").replace("<"," ").replace(">"," ").split(" ")
    num = np.mean([float(s) for s in split if s.replace(".", "",1).isdigit()])
    if "minutes" in lower or "minute" in lower or "min" in lower or "mins" in lower or "m " in lower:
    #if "m " in split:
        t = 60
    elif "hours" in lower or "hour" in lower or "hrs" in lower or "hr" in lower or "h " in lower:
    #elif "h " in split:
        t = 3600
    #times.append(num*t)
    if np.isnan(num)==False:
        return num*t
    if np.isnan(num)==True:
        return t

In [177]:
#twice as fast as my code :(
def infer_duration_in_seconds(text):
    # try different regexps to extract the total seconds
    metric_text = ["hour","minute","second","sec","segundo","min","hr"," m"," h"," s"]
    metric_seconds = [3600,60,1,1,1,60,3600,60,3600,1]
    text_replaced = text.replace(":"," min ").replace("+","").replace("-"," ").lower() #check for colon
    
    #digit and metric together
    for metric,mult in zip(metric_text,metric_seconds):
        regex = "\s*(\d+)\+?\s*{}s?".format(metric)
        res = re.findall(regex,text_replaced)
        if len(res)>0:
            return int(float(res[0]) * mult)
        
    #only metric
    for metric,mult in zip(metric_text,metric_seconds):
        metric_regex = "\s*\+?\s*{}\s?".format(metric)
        res = re.findall(metric_regex,text_replaced)
        if len(res)>0:
            return mult
    
    #only digit (seconds)
    dig_regex = "\s*(\d+)\+?\s*?"
    res = re.findall(dig_regex,text_replaced)
    if len(res)>0:
        return int(float(res[0]) * 1)
    
    return None

In [178]:
t = time.time()
df['duration_seconds'] = df['duration'].map(lambda x: infer_duration_in_seconds(x))
print time.time() - t

4.14482498169


In [179]:
df.isnull().sum()

city                   0
title                  0
shape                  0
duration               0
state                  0
link                   0
date                   0
date_datetime        242
duration_seconds    6306
geo                    0
dtype: int64

###Link col

In [202]:
df['link_no_ext'] = df['link'].map(lambda x: x[:-5])

###Geocode

In [69]:
#full df
df = pd.read_csv("ufo_nosummary.csv")
del df["Unnamed: 0"]

#unique geocodes
df_geocodes = pd.read_csv("geocodes_8973.csv")
del df_geocodes['Unnamed: 0']
df_geocodes.drop_duplicates(subset=['city','state'],inplace=True)
df_geocodes = df_geocodes.reset_index(drop=True)

#split lat/long
df_test = pd.DataFrame(df_geocodes.latlong.str.split(',',1).tolist(),
                                   columns = ['lat','long'])
df_test['lat'] = df_test['lat'].map(lambda x: x[1:])
df_test['long'] = df_test['long'].map(lambda x: x[:-1])
del df_geocodes['latlong']

#unique geocodes with lat/long separated
df_5359_unique = pd.concat([df_geocodes,df_test],axis=1)

#full df merged with unique geocodes, geocodes 56476!
df_56476 = pd.merge(df,df_5359_unique,how='inner',on=['city','state'])

#full df merged with unique geocodes, geocodes includes NaNs
df_56476_full = pd.merge(df,df_5359_unique,how='outer',on=['city','state'])

###See df

###To csv

In [84]:
#full
len(df_56476_full)

99477

In [75]:
#geocoded
len(df_56476)

56476

In [115]:
#not geocoded
#where both city and state are complete
len(not_yet_geocoded)

19120

In [99]:
city_states_notgeo = df_56476_full[df_56476_full['lat'].isnull()][['city','state']].drop_duplicates()
len(city_states_notgeo)

23155

In [106]:
not_yet_geocoded = city_states_notgeo[city_states_notgeo['city'].isnull()==False].reset_index(drop=True)
not_yet_geocoded = not_yet_geocoded[not_yet_geocoded['state'].isnull()==False].reset_index(drop=True)

In [114]:
not_yet_geocoded.to_csv("19120_not_geocoded.csv",encoding='utf-8')